In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Subset, Dataset
import torch.nn.functional as F
from transformers import AutoTokenizer
from tqdm.notebook import tqdm
import random
import re
import math
import os
import json
import types
import time
from sklearn.model_selection import train_test_split
# from GRU_collection import AdvancedWeatherGRU as WeatherTextGRU
from GRU_collection import AttentionWeatherGRU as WeatherTextGRU

def skip_only_model_special_tokens(tokens, tokenizer):
    # IDs of tokens to skip
    tokens_to_skip = set([
        tokenizer.cls_token_id,
        tokenizer.sep_token_id,
        tokenizer.pad_token_id
    ])
    
    # Filter out only model special tokens
    filtered_tokens = [t for t in tokens if t.item() not in tokens_to_skip]
    
    # Return as tensor
    return torch.tensor(filtered_tokens)

# WeatherDataset class for processing weather data
class WeatherDataset(Dataset):
    def __init__(self, weather_data, max_length=100):
        self.data = [weather_data] if isinstance(weather_data, dict) else weather_data
        self.max_length = max_length
        
        # Initialize wind directions from all data points
        self.wind_directions = sorted(list(set([d for data in self.data for d in data['windrichtung']])))
        self.wind_dir_to_idx = {d: i for i, d in enumerate(self.wind_directions)}
        
        # Calculate feature dimension - REMOVED rain_risk, rain_amount, wind_speed, and pressure
        self.feature_dim = (
            1 +  # temperature
            # REMOVED: 1 +  # rain risk
            # REMOVED: 1 +  # rain amount
            # REMOVED: 1 +  # wind speed
            # REMOVED: 1 +  # pressure
            1 +  # humidity
            1 +  # cloudiness
            len(self.wind_directions) +  # one-hot wind directions
            2 +  # time encoding (sin, cos)
            3 +  # sun features
            1    # sun hours
        )
        
    def replace_dates(self, text: str) -> str:
        text = re.sub(r"\b\d{1,2}\.\d{1,2}\.\d{4}\b", "<date>", text)
        return text
    
    def replace_city_and_units(self, text: str, city: str) -> str:
        text = re.sub(city, '<city>', text)
        unit_patterns = [
        # TEMPERATURE
        (r'(°[ ]*C|Grad)', ' <temp>'),
        # VELOCITY
        (r'[ ]*km/h', ' <velocity>'),
        # PERCENTILE
        (r'[ ]*%', ' <percentile>'),
        # RAINFALL DELTA
        (r'[ ]*l\\/m²', ' <rainfall>')
        ]
        for pattern, replacement in unit_patterns:
            try:
                text = re.sub(pattern, replacement, text)
            except Exception:
                continue

        # REMOVE MARKUP
        text = re.sub(r'\**', '', text)

        # REMOVE WEIRD PUNCUATION
        text = re.sub(r' \.', '.', text)

        # REMOVE UNNECESSARY NEWLINES
        text = re.sub(r'\n\n', '\n', text)

        # REMOVE SPACE AFTER NEWLINE
        text = re.sub(r'\n ', '\n', text)

        # REPLACE MULTIPLE WHITESPACES WITH ONE
        text = re.sub(r' +', ' ', text)
        return text

    def _parse_time(self, time_str):
        """Parse time string and handle missing data"""
        if time_str == '-' or not time_str:
            return None
        try:
            # Handle "HH:MM Uhr" format
            if ':' in time_str:
                hour, minute = map(int, time_str.split(' ')[0].split(':'))
                return hour + minute/60
            return None
        except (ValueError, IndexError):
            return None

    def _encode_time(self, time_str):
        # Convert "HH - HH Uhr" to cyclic features
        try:
            start_hour = int(time_str.split(' - ')[0])
            hour_sin = torch.sin(torch.tensor(2 * math.pi * start_hour / 24))
            hour_cos = torch.cos(torch.tensor(2 * math.pi * start_hour / 24))
            return torch.tensor([hour_sin, hour_cos])
        except (ValueError, IndexError):
            # Return neutral values for invalid time
            return torch.tensor([0.0, 1.0])
        
    def __len__(self):
        return len(self.data)
    
    def _encode_sun_info(self, sunrise, sunset, current_time):
        # Parse times, handling missing data
        sunrise_hour = self._parse_time(sunrise)
        sunset_hour = self._parse_time(sunset)
        
        try:
            current_hour = float(current_time.split(' - ')[0])
        except (ValueError, IndexError):
            # Return default values if current time is invalid
            return torch.tensor([0.0, 0.0, 0.0])
        
        # If sunrise or sunset is missing, use approximate values based on season
        if sunrise_hour is None or sunset_hour is None:
            # Return default encoding indicating uncertainty
            return torch.tensor([
                0.5,  # Unknown daylight status
                0.0,  # Neutral time since sunrise
                0.0   # Neutral time until sunset
            ])
        
        # Calculate daylight features
        is_daylight = (current_hour >= sunrise_hour) and (current_hour <= sunset_hour)
        
        if is_daylight:
            time_since_sunrise = (current_hour - sunrise_hour) / (sunset_hour - sunrise_hour)
            time_until_sunset = (sunset_hour - current_hour) / (sunset_hour - sunrise_hour)
        else:
            if current_hour < sunrise_hour:
                time_since_sunrise = -1 * (sunrise_hour - current_hour) / (24 - sunset_hour + sunrise_hour)
                time_until_sunset = -1
            else:
                time_since_sunrise = -1
                time_until_sunset = -1 * (current_hour - sunset_hour) / (24 - sunset_hour + sunrise_hour)
        
        return torch.tensor([float(is_daylight), time_since_sunrise, time_until_sunset])

    def one_hot_wind(self, wind_dir):
        encoding = torch.zeros(len(self.wind_directions))
        encoding[self.wind_dir_to_idx[wind_dir]] = 1
        return encoding
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Get sequence length from the data
        seq_len = len(item['temperatur_in_deg_C'])
        
        # Initialize features tensor with correct shape
        features = torch.zeros((seq_len, self.feature_dim))
        
        # Fill features one by one, maintaining consistent shapes
        current_idx = 0
        
        # Temperature feature
        features[:, current_idx] = torch.tensor([float(t) for t in item['temperatur_in_deg_C']])
        current_idx += 1
        
        # REMOVED: Rain risk feature
        # features[:, current_idx] = torch.tensor([float(r) for r in item['niederschlagsrisiko_in_perc']])
        # current_idx += 1
        
        # REMOVED: Rain amount feature
        # rain_values = []
        # last_valid = 0.0
        # for r in item['niederschlagsmenge_in_l_per_sqm']:
        #     try:
        #         if isinstance(r, str):
        #             val = float(r)
        #         elif isinstance(r, float) and not math.isnan(r):
        #             val = r
        #         else:
        #             val = last_valid
        #         rain_values.append(val)
        #         last_valid = val
        #     except ValueError:
        #         rain_values.append(last_valid)
        # features[:, current_idx] = torch.tensor(rain_values)
        # current_idx += 1
        
        # REMOVED: Wind speed feature
        # features[:, current_idx] = torch.tensor([float(w) for w in item['windgeschwindigkeit_in_km_per_s']])
        # current_idx += 1
        
        # REMOVED: Pressure feature
        # features[:, current_idx] = torch.tensor([float(p) for p in item['luftdruck_in_hpa']])
        # current_idx += 1
        
        # Humidity feature
        features[:, current_idx] = torch.tensor([float(h) for h in item['relative_feuchte_in_perc']])
        current_idx += 1
        
        # Cloudiness feature
        features[:, current_idx] = torch.tensor([float(c.split('/')[0]) / 8 for c in item['bewölkungsgrad']])
        current_idx += 1
        
        # Wind directions (one-hot encoded)
        wind_features = torch.stack([self.one_hot_wind(w) for w in item['windrichtung']])
        features[:, current_idx:current_idx + len(self.wind_directions)] = wind_features
        current_idx += len(self.wind_directions)
        
        # Time features
        time_features = torch.stack([self._encode_time(t) for t in item['times']])
        features[:, current_idx:current_idx + 2] = time_features
        current_idx += 2
        
        # Sun features
        sun_features = torch.stack([
            self._encode_sun_info(
                item.get('sunrise', '-'), 
                item.get('sundown', '-'), 
                t
            ) for t in item['times']
        ])
        features[:, current_idx:current_idx + 3] = sun_features
        current_idx += 3
        
        # Sun hours feature
        sun_hours = torch.tensor([1.0 if "fast nicht zu sehen" in item.get('sunhours', '') else 0.0])
        features[:, current_idx] = sun_hours.expand(seq_len)
        
        return {
            'features': features,
            'text': self.replace_dates(self.replace_city_and_units(item['gpt_rewritten_apokalyptisch_v2'], item['city']))
        }

def create_improved_dataloader(dataset, batch_size, tokenizer):
    """
    Creates a more efficient DataLoader with dynamic sequence length handling
    """
    def collate_fn(batch_list):
        # Extract features and texts
        features = torch.stack([item['features'] for item in batch_list])
        texts = [item['text'] for item in batch_list]
        
        # Normalize features within batch for better training stability
        features = (features - features.mean(dim=(0, 1), keepdim=True)) / (
            features.std(dim=(0, 1), keepdim=True) + 1e-8)
        
        # Get token IDs with padding
        encoded = tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors='pt'
        )
        
        # Get sequence lengths for potential packed sequences
        seq_lengths = (encoded['attention_mask'] == 1).sum(dim=1)
        
        return {
            'features': features,
            'text': encoded['input_ids'],
            'attention_mask': encoded['attention_mask'],
            'seq_lengths': seq_lengths
        }
    
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn
    )

def count_model_parameters(model):
    """Count the number of trainable parameters in the model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def set_seed(seed):
    """Set random seed for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def prepare_for_improved_training(dataset):
    """Prepare the dataset for improved training"""
    # Split into train/validation
    indices = list(range(len(dataset)))
    train_indices, val_indices = train_test_split(
        indices, test_size=0.1, random_state=42
    )
    
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)
    
    return train_dataset, val_dataset


import concurrent.futures
import torch
from torch.utils.data import Subset
from tqdm.notebook import tqdm
import numpy as np
from functools import partial

def process_item(idx, dataset):
    """Process a single dataset item and validate it"""
    try:
        sample = dataset[idx]
        features = sample['features']
        
        # Check for NaN or Inf values
        if torch.isnan(features).any() or torch.isinf(features).any():
            return None
        
        return idx
    except Exception as e:
        return None

def validate_and_clean_data_multithreaded(dataset, num_workers=None):
    """Validate dataset and remove samples with NaN or Inf values using multiple threads"""
    # If num_workers is not specified, use CPU count
    if num_workers is None:
        import multiprocessing
        num_workers = max(1, multiprocessing.cpu_count() - 1)  # Leave one CPU free
    
    print(f"Validating data using {num_workers} workers...")
    
    # Use partial to create a function with the dataset already bound
    process_fn = partial(process_item, dataset=dataset)
    
    valid_indices = []
    total_items = len(dataset)
    
    # Use ThreadPoolExecutor for I/O bound operations
    with tqdm(total=total_items, desc="Validating items") as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            # Submit all tasks
            future_to_idx = {executor.submit(process_fn, idx): idx for idx in range(total_items)}
            
            # Process results as they complete
            for future in concurrent.futures.as_completed(future_to_idx):
                idx = future_to_idx[future]
                try:
                    result = future.result()
                    if result is not None:
                        valid_indices.append(result)
                except Exception as e:
                    print(f"Error processing item {idx}: {e}")
                
                pbar.update(1)
    
    invalid_count = total_items - len(valid_indices)
    print(f"Found {invalid_count} invalid samples out of {total_items}")
    print(f"Keeping {len(valid_indices)} valid samples")
    
    # Create a new subset with valid samples
    return Subset(dataset, valid_indices)

def load_and_validate_data_multithreaded(data_path='data/files_for_chatGPT/2024-12-12/', num_workers=None):
    """Load and validate weather data from JSON files using multiple threads"""
    import os
    import json
    
    # If num_workers is not specified, use CPU count
    if num_workers is None:
        import multiprocessing
        num_workers = max(1, multiprocessing.cpu_count() - 1)  # Leave one CPU free
    
    # Check if we're in the right directory, navigate if needed
    if not os.path.exists(data_path):
        base_paths = ['.', '..', '../..']
        for base in base_paths:
            test_path = os.path.join(base, data_path)
            if os.path.exists(test_path):
                data_path = test_path
                break
    
    # List JSON files
    files = [f for f in os.listdir(data_path) if f.endswith('.json')]
    print(f"Found {len(files)} JSON files")
    
    # Define function to process a single file
    def process_file(file):
        file_path = os.path.join(data_path, file)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
            # Validate file
            if not {'gpt_rewritten_apokalyptisch_v2', 'city'}.issubset(data.keys()):
                return None
                
            if not isinstance(data['city'], str) or not data['city'].strip():
                return None
                
            if not isinstance(data['gpt_rewritten_apokalyptisch_v2'], str) or not data['gpt_rewritten_apokalyptisch_v2'].strip():
                return None
            
            # File is valid, return with a key
            key = (file.split('-')[-1]).split('_')[0]
            return (key, data)
        except json.JSONDecodeError:
            return None
        except Exception as e:
            print(f"Error processing {file}: {e}")
            return None
    
    data_dict = {}
    
    # Process files in parallel
    print(f"Loading files using {num_workers} workers...")
    with tqdm(total=len(files), desc="Loading files") as pbar:
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            # Submit all tasks
            futures = {executor.submit(process_file, file): file for file in files}
            
            # Process results as they complete
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                if result is not None:
                    key, data = result
                    data_dict[key] = data
                pbar.update(1)
    
    # Convert to list
    return list(data_dict.values())

# Combined function to load, create, and clean dataset in parallel
def prepare_dataset_multithreaded(num_workers=None):
    """Full pipeline to load, create, and clean the dataset using multiple threads"""
    print("Starting multithreaded dataset preparation...")
    start_time = time.time()
    
    # Load data
    weather_data = load_and_validate_data_multithreaded(num_workers=num_workers)
    print(f"Loaded {len(weather_data)} weather data points")
    
    # Create dataset
    dataset = WeatherDataset(weather_data)
    
    # Clean dataset
    clean_dataset = validate_and_clean_data_multithreaded(dataset, num_workers=num_workers)
    
    # Split into train/validation
    train_dataset, val_dataset = prepare_for_improved_training(clean_dataset)
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    
    end_time = time.time()
    print(f"Dataset preparation completed in {end_time - start_time:.2f} seconds")
    
    return clean_dataset, train_dataset, val_dataset
# Set random seed
set_seed(42)
dataset_start = time.time()

# Use multithreaded dataset preparation
clean_dataset, train_dataset, val_dataset = prepare_dataset_multithreaded()

dataset_end = time.time()
print(f"Dataset preparation completed in {dataset_end - dataset_start:.2f} seconds")

Starting multithreaded dataset preparation...
Found 28812 JSON files
Loading files using 11 workers...


Loading files:   0%|          | 0/28812 [00:00<?, ?it/s]

Loaded 19358 weather data points
Validating data using 11 workers...


Validating items:   0%|          | 0/19358 [00:00<?, ?it/s]

Found 41 invalid samples out of 19358
Keeping 19317 valid samples
Training dataset size: 17385
Validation dataset size: 1932
Dataset preparation completed in 127.77 seconds
Dataset preparation completed in 127.78 seconds


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, BertTokenizer
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm
import random
import numpy as np
import time
import math
import os
import json
from collections import Counter

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def weather_collate_fn(batch_list, tokenizer):
    # Extract features and texts
    features = torch.stack([item['features'] for item in batch_list])
    texts = [item['text'] for item in batch_list]
    
    # Normalize features within batch for better training stability
    features = (features - features.mean(dim=(0, 1), keepdim=True)) / (
        features.std(dim=(0, 1), keepdim=True) + 1e-8)
    
    # Tokenize with padding
    encoded = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,  # Limit sequence length for efficiency
        return_tensors='pt'
    )
    
    return {
        'features': features,
        'text': encoded['input_ids'],
        'attention_mask': encoded['attention_mask']
    }

def reduce_vocabulary(tokenizer, full_dataset, batch_size=64):
    """Identify used tokens and create a reduced vocabulary mapping"""
    print("Analyzing vocabulary usage to reduce model size...")
    
    # Count tokens
    token_counter = Counter()
    
    """Count tokens from the entire dataset"""
    print("Analyzing vocabulary usage to reduce model size...")
    token_counter = Counter()
    
    # Process the entire dataset directly without DataLoader
    for idx in tqdm(range(len(full_dataset)), desc="Scanning token usage"):
        # Get the raw text directly (handles Subset objects correctly)
        if isinstance(full_dataset, Subset):
            sample = full_dataset.dataset[full_dataset.indices[idx]]
        else:
            sample = full_dataset[idx]
        
        text = sample['text']
        
        # Tokenize directly
        tokens = tokenizer.encode(text, add_special_tokens=True)
        token_counter.update(tokens)
    
    # Always keep special tokens
    for special_token in tokenizer.special_tokens_map.values():
        if isinstance(special_token, str):
            token_id = tokenizer.convert_tokens_to_ids(special_token)
            if token_id not in token_counter:
                token_counter[token_id] = 1
        elif isinstance(special_token, list):
            for token in special_token:
                token_id = tokenizer.convert_tokens_to_ids(token)
                if token_id not in token_counter:
                    token_counter[token_id] = 1
    
    # Sort by frequency for efficient token ID assignment
    used_token_ids = sorted(token_counter.keys())
    
    # Create token ID mapping (old ID -> new ID)
    token_id_map = {old_id: new_id for new_id, old_id in enumerate(used_token_ids)}
    
    # Create reverse mapping for inference
    reverse_token_id_map = {new_id: old_id for old_id, new_id in token_id_map.items()}
    
    # Store the mappings for later use
    token_mappings = {
        'token_id_map': token_id_map,
        'reverse_token_id_map': reverse_token_id_map,
        'used_token_ids': used_token_ids
    }
    
    # Update vocabulary size to reduced size
    reduced_vocab_size = len(used_token_ids)
    original_vocab_size = len(tokenizer.vocab)
    print(f"Reduced vocabulary from {original_vocab_size:,} to {reduced_vocab_size:,} tokens " 
          f"({reduced_vocab_size/original_vocab_size*100:.1f}%)")
    
    return token_mappings, reduced_vocab_size

# Map tokens function for data loaders
def map_tokens_fn(batch, token_id_map):
    # Map the token IDs to new IDs
    old_tokens = batch['text']
    new_tokens = torch.zeros_like(old_tokens)
    
    # Apply mapping
    for i in range(old_tokens.size(0)):
        for j in range(old_tokens.size(1)):
            old_id = old_tokens[i, j].item()
            new_tokens[i, j] = token_id_map.get(old_id, 0)  # Default to 0 if token not found
    
    batch['text'] = new_tokens
    return batch

def train_model(args):
    global tokenizer  # Make tokenizer accessible to model
    
    print(f"Using device: {device}")
    
    # Set seed for reproducibility
    torch.manual_seed(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    
    # Load tokenizer - using German BERT for German text
    print("Loading tokenizer...")
    tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
    
    # Add special tokens that appear in our texts
    special_tokens = {'additional_special_tokens': ['<city>', '<temp>']}
    tokenizer.add_special_tokens(special_tokens)
    
    print("Preparing datasets...")
    start_time = time.time()

    print(f"Dataset preparation completed in {time.time() - start_time:.2f}s")
    
    # Reduce vocabulary
    token_mappings, reduced_vocab_size = reduce_vocabulary(tokenizer, clean_dataset, args.batch_size)
    
    # Create dataloaders with token mapping
    train_loader = DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        collate_fn=lambda batch: map_tokens_fn(weather_collate_fn(batch, tokenizer), token_mappings['token_id_map'])
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=lambda batch: map_tokens_fn(weather_collate_fn(batch, tokenizer), token_mappings['token_id_map'])
    )
    
    # Get feature dimension
    feature_dim = clean_dataset.dataset.feature_dim
    
    print(f"Feature dimension: {feature_dim}")
    print(f"Reduced vocabulary size: {reduced_vocab_size}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Create model
    print("Creating model...")
    model = WeatherTextGRU(
        feature_dim=feature_dim,
        hidden_size=args.hidden_size,
        vocab_size=reduced_vocab_size,
        dropout=args.dropout
    ).to(device)
    
    # Count parameters
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Model has {num_params:,} trainable parameters")
    
    # Ensure we're under the 3M parameter limit
    if num_params > 30_000_000:
        print(f"WARNING: Model exceeds 3M parameters ({num_params:,}), reducing hidden size...")
        
        # Reduce hidden size until we're under 3M params
        while num_params > 30_000_000 and args.hidden_size > 128:
            args.hidden_size -= 32
            
            model = WeatherTextGRU(
                feature_dim=feature_dim,
                hidden_size=args.hidden_size,
                vocab_size=reduced_vocab_size,
                dropout=args.dropout
            ).to(device)
            
            num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Adjusted model: {num_params:,} parameters with hidden_size={args.hidden_size}")
    
    # Setup aggressive optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=args.lr,
        weight_decay=0.01,
        betas=(0.9, 0.999)
    )
    
    # Loss function (with 'none' reduction for masking)
    criterion = nn.CrossEntropyLoss(reduction='none')
    
    # Or try focal loss to focus on harder examples
    def focal_loss(predictions, targets, gamma=2.0, alpha=0.25):
        ce_loss = F.cross_entropy(predictions, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = alpha * (1-pt)**gamma * ce_loss
        return focal_loss

    # Learning rate scheduler for aggressive learning
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=args.lr * 10,  # Peak at 10x the base learning rate
        total_steps=args.epochs * len(train_loader),
        pct_start=0.1,  # Aggressive warm-up
        div_factor=25.0,  # Determines initial lr
        final_div_factor=10000.0,  # For steep decay at the end
        anneal_strategy='cos'
    )
    
    # Training parameters
    best_val_loss = float('inf')
    best_train_loss = float('inf')
    patience_counter = 0
    clip_value = 1.0
    
    # Prepare save directory
    os.makedirs(args.save_dir, exist_ok=True)
    
    training_start = time.time()
    
    # Training loop
    for epoch in range(args.epochs):
        epoch_start = time.time()
        print(f"\nEpoch {epoch+1}/{args.epochs}")
        
        # Training
        model.train()
        train_loss = 0.0
        
        train_pbar = tqdm(train_loader, desc="Training")
        for batch in train_pbar:
            # Move data to device
            features = batch['features'].to(device)
            tokens = batch['text'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Forward pass with very aggressive teacher forcing
            outputs = model(features, tokens, teacher_forcing_ratio=0.8)
            
            # Calculate loss (excluding first token - CLS)
            output_flat = outputs[:, 1:].reshape(-1, outputs.shape[-1])
            target_flat = tokens[:, 1:].reshape(-1)
            mask_flat = attention_mask[:, 1:].reshape(-1)
            
            # Compute token-wise loss
            losses = criterion(output_flat, target_flat)
            
            # Apply mask to ignore padding tokens
            masked_losses = losses * mask_flat
            
            # Average over non-padding tokens
            batch_loss = masked_losses.sum() / (mask_flat.sum() + 1e-8)
            
            # Backward pass
            optimizer.zero_grad()
            batch_loss.backward()
            
            # Clip gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            
            # Update weights
            optimizer.step()
            
            # Update learning rate
            scheduler.step()
            
            # Update metrics
            train_loss += batch_loss.item()
            train_pbar.set_postfix({"loss": f"{batch_loss.item():.4f}"})
        
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc="Validating")
            for batch in val_pbar:
                features = batch['features'].to(device)
                tokens = batch['text'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                # Forward pass (no teacher forcing)
                outputs = model(features, tokens, teacher_forcing_ratio=0.0)
                
                # Calculate loss
                output_flat = outputs[:, 1:].reshape(-1, outputs.shape[-1])
                target_flat = tokens[:, 1:].reshape(-1)
                mask_flat = attention_mask[:, 1:].reshape(-1)
                
                losses = criterion(output_flat, target_flat)
                masked_losses = losses * mask_flat
                batch_loss = masked_losses.sum() / (mask_flat.sum() + 1e-8)
                
                val_loss += batch_loss.item()
                val_pbar.set_postfix({"loss": f"{batch_loss.item():.4f}"})
        
        avg_val_loss = val_loss / len(val_loader)
        # After computing train_loss for each epoch
        avg_train_loss = train_loss / len(train_loader)  
        avg_val_loss = val_loss / len(val_loader)

        epoch_time = time.time() - epoch_start
        print(f"Epoch {epoch+1} completed in {epoch_time:.2f}s")
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss

        # Save model and token mappings
        if avg_train_loss < best_train_loss:
            best_train_loss = avg_train_loss
            # Save model and token mappings
            model_path = os.path.join(args.save_dir, "best_weather_text_model.pt")
            torch.save({
                'model_state_dict': model.state_dict(),
                'token_mappings': token_mappings,
                'model_config': {
                    'feature_dim': feature_dim,
                    'hidden_size': args.hidden_size,
                    'vocab_size': reduced_vocab_size,
                    'dropout': args.dropout
                },
                'train_args': vars(args),
                'epoch': epoch,
                'val_loss': best_val_loss
            }, model_path)
            
            print(f"✓ Model saved to {model_path}!")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= args.patience:
                print(f"Early stopping triggered after {epoch+1} epochs")
                break
        
        # Generate sample text periodically
        if (epoch + 1) % 3 == 0 or epoch == 0 or epoch == args.epochs - 1:
            generate_samples(model, tokenizer, val_loader, token_mappings, 1)
    
    total_time = time.time() - training_start
    print(f"\nTraining completed in {total_time:.2f}s")
    
    # Load best model for final evaluation
    checkpoint = torch.load(os.path.join(args.save_dir, "best_weather_text_model.pt"))
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Final evaluation
    model.eval()
    final_val_loss = 0.0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Final Evaluation"):
            features = batch['features'].to(device)
            tokens = batch['text'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(features, tokens, teacher_forcing_ratio=0.0)
            
            output_flat = outputs[:, 1:].reshape(-1, outputs.shape[-1])
            target_flat = tokens[:, 1:].reshape(-1)
            mask_flat = attention_mask[:, 1:].reshape(-1)
            
            losses = criterion(output_flat, target_flat)
            masked_losses = losses * mask_flat
            batch_loss = masked_losses.sum() / (mask_flat.sum() + 1e-8)
            
            final_val_loss += batch_loss.item()
    
    final_val_loss /= len(val_loader)
    print(f"Final validation loss: {final_val_loss:.4f}")
    
    # Generate final samples
    print("\nFinal generated samples:")
    generate_samples(model, tokenizer, val_loader, token_mappings, args.sample_count)
    
    return model, tokenizer, token_mappings

def generate_samples(model, tokenizer, data_loader, token_mappings, num_samples=5):
    """Generate text samples from the model"""
    model.eval()
    
    # Get mapping for converting back to original token IDs
    reverse_map = token_mappings['reverse_token_id_map']
    
    # IDs of tokens to keep (exclude [PAD], [CLS], [SEP])
    tokens_to_exclude = {
        tokenizer.pad_token_id,
        tokenizer.cls_token_id, 
        tokenizer.sep_token_id
    }

    # Get samples
    samples = []
    data_iter = iter(data_loader)
    
    for _ in range(num_samples):
        try:
            sample_batch = next(data_iter)
            samples.append(sample_batch)
        except StopIteration:
            # Reset iterator if we run out of samples
            data_iter = iter(data_loader)
            sample_batch = next(data_iter)
            samples.append(sample_batch)
    
    # Generate text for each sample
    for i, batch in enumerate(samples):
        sample_idx = 0  # Just use the first item in the batch
        
        # Get features and original tokens
        sample_features = batch['features'][sample_idx].unsqueeze(0).to(device)
        sample_tokens_mapped = batch['text'][sample_idx]
        
        # Get temperature, humidity and cloudiness data for reference
        temp_values = sample_features[0, :, 0].cpu().numpy()  # Temperature (first feature)
        humidity_values = sample_features[0, :, 1].cpu().numpy()  # Humidity (second feature)
        cloud_values = sample_features[0, :, 2].cpu().numpy()  # Cloudiness (third feature)
        
        # Generate text
        with torch.no_grad():
            generated_output = model(sample_features)
        
        # Get token predictions
        # Replace your existing multinomial sampling code with this:
        # generated_tokens_mapped = torch.zeros(generated_output[0].size(0), dtype=torch.long, device=device)

        # # Apply nucleus sampling for each position in the sequence
        # for pos in range(generated_output[0].size(0)):
        #     logits = generated_output[0][pos]  # Get logits for this position
        #     generated_tokens_mapped[pos] = nucleus_sampling(logits, p=0.9)
        # temperature = 0.2  # Adjust between 0.5-1.0 for creativity vs coherence
        # probs = F.softmax(generated_output[0] / temperature, dim=1)
        # generated_tokens_mapped = torch.multinomial(probs, num_samples=1).squeeze(-1)
        generated_tokens_mapped = generated_output[0].argmax(dim=1)
        
        # Map tokens back to original vocabulary
        original_tokens = torch.tensor([
            reverse_map[token.item()] for token in sample_tokens_mapped
        ])
        
        generated_tokens = torch.tensor([
            reverse_map[token.item()] for token in generated_tokens_mapped
        ])
        
        # Filter out unwanted special tokens from generation
        filtered_generated = [token.item() for token in generated_tokens 
                           if token.item() not in tokens_to_exclude]
        
        # # Start text with "In" if it doesn't already
        # if filtered_generated and tokenizer.decode([filtered_generated[0]]) != "In":
        #     in_token_id = tokenizer.convert_tokens_to_ids("In")
        #     filtered_generated = [in_token_id] + filtered_generated
        
        # Decode to text
        original_text = tokenizer.decode(original_tokens, skip_special_tokens=False)
        generated_text = tokenizer.decode(filtered_generated, skip_special_tokens=False)
        
        print(f"\nSample {i+1}:")
        print(f"Temperature: {[round(float(t), 1) for t in temp_values]}")
        print(f"Humidity: {[round(float(h), 1) for h in humidity_values]}")
        print(f"Cloudiness: {[round(float(c), 2) for c in cloud_values]}")
        print(f"Original: {original_text}")
        print(f"Generated: {generated_text}")
        print("-" * 80)

def generate_weather_text(model, tokenizer, features, token_mappings):
    """Generate a weather text description from features"""
    model.eval()
    
    # IDs of tokens to exclude
    tokens_to_exclude = {
        tokenizer.pad_token_id,
        tokenizer.cls_token_id, 
        tokenizer.sep_token_id
    }
    
    with torch.no_grad():
        # Normalize features
        features = (features - features.mean(dim=0, keepdim=True)) / (
            features.std(dim=0, keepdim=True) + 1e-8)
        
        # Add batch dimension and move to device
        features = features.unsqueeze(0).to(device)
        
        # Generate text
        outputs = model(features)
        
        # Using temperature sampling for more diverse outputs
        temperature = 1.0
        probs = F.softmax(outputs[0] / temperature, dim=1)
        generated_tokens_mapped = torch.multinomial(probs, num_samples=1).squeeze(-1)
        
        # Map tokens back to original vocabulary
        reverse_map = token_mappings['reverse_token_id_map']
        generated_tokens = [reverse_map[token.item()] for token in generated_tokens_mapped]
        
        # Filter out unwanted special tokens
        filtered_tokens = [token for token in generated_tokens 
                          if token not in tokens_to_exclude]
        
        # Start text with "In" if it doesn't already
        if filtered_tokens and tokenizer.decode([filtered_tokens[0]]) != "In":
            in_token_id = tokenizer.convert_tokens_to_ids("In")
            filtered_tokens = [in_token_id] + filtered_tokens
        
        # Decode to text
        generated_text = tokenizer.decode(filtered_tokens, skip_special_tokens=False)
        
        return generated_text

if __name__ == "__main__":
    import argparse
    torch.cuda.empty_cache()
    # Parse command line arguments
    # parser = argparse.ArgumentParser(description='Train a weather text generation model')
    # parser.add_argument('--epochs', type=int, default=30, help='Number of training epochs')
    # parser.add_argument('--batch_size', type=int, default=64, help='Batch size for training')
    # parser.add_argument('--hidden_size', type=int, default=256, help='Hidden size for model')
    # parser.add_argument('--lr', type=float, default=1e-3, help='Initial learning rate')
    # parser.add_argument('--dropout', type=float, default=0.2, help='Dropout rate')
    # parser.add_argument('--patience', type=int, default=5, help='Early stopping patience')
    # parser.add_argument('--save_dir', type=str, default='./models', help='Directory to save models')
    # parser.add_argument('--seed', type=int, default=42, help='Random seed')
    # parser.add_argument('--test_only', action='store_true', help='Run only inference on test samples')
    # parser.add_argument('--sample_count', type=int, default=5, help='Number of samples to generate')
    # parser.add_argument('--model_path', type=str, default=None, help='Path to load model from')
    # args = parser.parse_args()

    from types import SimpleNamespace
    # Create arguments object manually
    args = SimpleNamespace(
        epochs=20,
        batch_size=128,
        hidden_size=256,
        lr=1e-3,
        dropout=0.3,
        patience=15,
        save_dir='./models',
        seed=42,
        test_only=False,
        sample_count=5,
        model_path=None
    )

    # Create save directory if it doesn't exist
    os.makedirs(args.save_dir, exist_ok=True)
    
    # Train or load model
    if not args.test_only:
        print(f"Training model with {args.epochs} epochs, batch size {args.batch_size}, hidden size {args.hidden_size}")
        model, tokenizer, token_mappings = train_model(args)
    else:
        # Load model for testing
        print("Loading pre-trained model for inference...")
        
        # Determine model path
        model_path = args.model_path or os.path.join(args.save_dir, "best_weather_text_model.pt")
        
        if not os.path.exists(model_path):
            print(f"Error: Model file not found at {model_path}")
            exit(1)
        
        # Load checkpoint
        checkpoint = torch.load(model_path, map_location=device)
        
        # Load tokenizer
        tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')
        special_tokens = {'additional_special_tokens': ['<city>', '<temp>']}
        tokenizer.add_special_tokens(special_tokens)
        
        # Get token mappings
        token_mappings = checkpoint['token_mappings']
        
        # Get model config
        model_config = checkpoint['model_config']
        
        # Create model
        model = WeatherTextGRU(
            feature_dim=model_config['feature_dim'],
            hidden_size=model_config['hidden_size'],
            vocab_size=model_config['vocab_size'],
            dropout=model_config.get('dropout', 0.2)
        ).to(device)
        
        # Load weights
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        print(f"Model loaded from {model_path}")
        print(f"Model has {sum(p.numel() for p in model.parameters() if p.requires_grad):,} parameters")
        
        # Prepare datasets for testing
        set_seed(args.seed)
        
        # Create dataloader with token mapping
        val_loader = DataLoader(
            val_dataset,
            batch_size=args.batch_size,
            shuffle=True,
            collate_fn=lambda batch: map_tokens_fn(
                weather_collate_fn(batch, tokenizer), 
                token_mappings['token_id_map']
            )
        )
        
        # Generate samples
        print(f"\nGenerating {args.sample_count} weather text samples:")
        generate_samples(model, tokenizer, val_loader, token_mappings, args.sample_count)
    
    print("Done!")

Training model with 20 epochs, batch size 128, hidden size 256
Using device: cuda
Loading tokenizer...
Preparing datasets...
Dataset preparation completed in 0.00s
Analyzing vocabulary usage to reduce model size...
Analyzing vocabulary usage to reduce model size...


Scanning token usage: 100%|██████████| 19317/19317 [00:59<00:00, 322.44it/s]


Reduced vocabulary from 30,000 to 8,158 tokens (27.2%)
Feature dimension: 18
Reduced vocabulary size: 8158
Training samples: 17385
Validation samples: 1932
Creating model...
Model has 8,319,262 trainable parameters

Epoch 1/20


Validating: 100%|██████████| 16/16 [00:11<00:00,  1.39it/s, loss=7.6513]


Epoch 1 completed in 526.69s
Train Loss: 5.6948 | Val Loss: 7.4181
✓ Model saved to ./models\best_weather_text_model.pt!

Sample 1:
Temperature: [0.3, 0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Humidity: [1.4, 1.3, 1.3, 1.2, 1.1, 1.3, 1.2, 1.1, 1.1, 1.1, 1.1, 1.3, 1.3, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2]
Cloudiness: [1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61]
Original: [CLS] In <city> bricht ein grauer Tag an. Der Himmel weint und die Wolken sind schwer. Regen prasselt nieder und die Temperaturen schwanken zwischen 19 und 20 <temp>. Menschen müssen sich schützen. Schirme sind unerlässlich. Sie können nicht zuhause bleiben. Wenn die Sonne sinkt, hält der Regen an. Die Nacht bringt keine Ruhe. Die Temperaturen fallen auf 19 <temp>, und das Wasser fließt unaufhörlich. Die Dunkelheit wird nass und küh

...

Epoch 20 completed in 577.53s
Train Loss: 2.9903 | Val Loss: 9.6044


C:\Users\franj\AppData\Local\Temp\ipykernel_12968\227350822.py:364: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(os.path.join(args.save_dir, "best_w


Sample 1:
Temperature: [0.3, 0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Humidity: [1.4, 1.3, 1.3, 1.2, 1.1, 1.3, 1.2, 1.1, 1.1, 1.1, 1.1, 1.3, 1.3, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2]
Cloudiness: [1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61]
Original: [CLS] In <city> bricht ein grauer Tag an. Der Himmel weint und die Wolken sind schwer. Regen prasselt nieder und die Temperaturen schwanken zwischen 19 und 20 <temp>. Menschen müssen sich schützen. Schirme sind unerlässlich. Sie können nicht zuhause bleiben. Wenn die Sonne sinkt, hält der Regen an. Die Nacht bringt keine Ruhe. Die Temperaturen fallen auf 19 <temp>, und das Wasser fließt unaufhörlich. Die Dunkelheit wird nass und kühl. Die Straßen spiegeln das Licht der Lampen. Stille herrscht, doch der Regen tobt weiter. In <city> ist der Tag von Trau

Final Evaluation: 100%|██████████| 16/16 [00:13<00:00,  1.20it/s]


Final validation loss: 9.6650

Final generated samples:

Sample 1:
Temperature: [0.3, 0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
Humidity: [1.4, 1.3, 1.3, 1.2, 1.1, 1.3, 1.2, 1.1, 1.1, 1.1, 1.1, 1.3, 1.3, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2]
Cloudiness: [1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61, 1.61]
Original: [CLS] In <city> bricht ein grauer Tag an. Der Himmel weint und die Wolken sind schwer. Regen prasselt nieder und die Temperaturen schwanken zwischen 19 und 20 <temp>. Menschen müssen sich schützen. Schirme sind unerlässlich. Sie können nicht zuhause bleiben. Wenn die Sonne sinkt, hält der Regen an. Die Nacht bringt keine Ruhe. Die Temperaturen fallen auf 19 <temp>, und das Wasser fließt unaufhörlich. Die Dunkelheit wird nass und kühl. Die Straßen spiegeln das Licht der Lampen. Stille herrscht, do

# BASIC Implementation
Params: 
batch:  128
hidden: 256
droput: 0.2
Best loss: 2.2737
8,161 tokens

CAP at 6 M Params
# Loss Function Modification with old params
- Using CrossEntropyLoss with label smoothing 0.1
- Using Focal loss with gama=2 and alpha=0.25
- Achieved Best Loss: 0.5061 (Focal Loss) with Val loss: 2.0764


# Loss Function Modification with new params
- Using CrossEntropyLoss with label smoothing 0.1
- Using Focal loss with gama=2 and alpha=0.25
- Droput: 0.3
- Batch: 32
- Hidden: 256
1.st run CAP at 6 M Params
- Best Loss: 0.5407 | Val Loss: 2.1299
2.nd run CAP at 10 M Params
- Not necessary to run again, the model is not doing better

# crossentropy with a lot of params
Train Loss: 2.5136 | Val Loss: 8.6019
Training model with 10 epochs, batch size 64, hidden size 512
Model has 17,277,537 trainable parameters
Training model with 10 epochs, batch size 64, hidden size 512
Sample 5:
Temperature: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5]
Humidity: [-0.3, -0.4, -0.5, -0.6, -0.6, -0.6, -0.7, -0.8, -0.8, -1.0, -1.2, -1.3, -1.3, -1.2, -1.0, -0.9, -0.7, 0.0, 0.3, 0.4, 0.3, -0.0, -0.4, -0.2]
Cloudiness: [-1.13, -1.13, -1.13, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, 0.25, -0.09, -0.44, -0.44, -0.09]
Original: [CLS] In <city> brennt die Sonne am Morgen und Nachmittag. Die Temperatur steigt auf schreckliche 26 <temp>. Die Hitze zieht durch die Straßen. Am Abend zieht eine dunkle Wolkendecke auf. Der Himmel weint trübe Gedanken. Die Temperatur bleibt bei 22 <temp>. Die Nacht bringt Schatten. Wolken verschlingen die Sterne. Ihre Lichtpunkte sind kaum noch zu sehen. Es ist, als ob die Dunkelheit alles verschluckt. Die Temperaturen sinken auf unbehagliche 23 <temp>. Ein unheilvolles Gefühl schleicht durch die Luft. Der Wind trägt leise Schreie mit sich. Der Tag endet im Zwielicht, während [SEP]
Generated: [PAD]lose Sonne am Tag. Die Temperaturen steigen auf 23 bis 36 <temp>. Die Luft flirrt vor Hitze, während die Hitze des Tages schwindet. In der Nacht wird der Himmel klar, doch die Dunkelheit bringt keine Erleichterung. Die Wolken bleiben wie ein schwerer Vorhang aus. Die Temperaturen sinken auf 23 <temp>. Die Nacht bringt keine Erleichterung. Die Stille der Nacht ist erdrückend, als ob die Welt in einen tiefen Schlaf gefallen ist. Kein Wind weht,
--------------------------------------------------------------------------------

Train Loss: 3.3150 | Val Loss: 8.1026
Training model with 10 epochs, batch size 32, hidden size 512
Model has 17,277,537 trainable parameters
Original: [CLS] In <city> erhebt sich der Morgen wie ein schwerer Schleier aus Wolken. Die Temperatur bleibt gefangen bei 5 <temp>. Mit dem Mittag zieht der Regen auf, als wären dunkle Mächte am Werk. Den ganzen Abend über wird das Land in Wasser getränkt, während die Werte zwischen 5 und 7 <temp> schwanken. Der Himmel weint und die Welt bleibt im Schatten. In der Nacht, wenn die Dunkelheit sich ausbreitet, wird der Regen nicht aufhören. Die Temperaturen sinken zurück auf 5 <temp>. Die Luft fühlt sich kalt und feucht an, als ob die Welt in einem ewigen Sturm gefangen ist. Ein unheilvolles [SEP]
Generated: [PAD] am Nachmittag übers. Die Temperaturen steigen von 13 <temp>. Die Luft ist schwer und bedrückt. Die Wolken hängen tief, als ob die Welt. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Kälte kriecht. Die Luft ist schwer. Die Luft ist schwer und bedrückend. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Temperaturen steigen
--------------------------------------------------------------------------------
Done!

Train Loss: 2.1875 | Val Loss: 9.0432
# BASIC Implementation
Params: 
batch:  128
hidden: 256
droput: 0.2
Best loss: 2.2737
8,161 tokens

CAP at 6 M Params
# Loss Function Modification with old params
- Using CrossEntropyLoss with label smoothing 0.1
- Using Focal loss with gama=2 and alpha=0.25
- Achieved Best Loss: 0.5061 (Focal Loss) with Val loss: 2.0764


# Loss Function Modification with new params
- Using CrossEntropyLoss with label smoothing 0.1
- Using Focal loss with gama=2 and alpha=0.25
- Droput: 0.3
- Batch: 32
- Hidden: 256
1.st run CAP at 6 M Params
- Best Loss: 0.5407 | Val Loss: 2.1299
2.nd run CAP at 10 M Params
- Not necessary to run again, the model is not doing better

# crossentropy with a lot of params
Train Loss: 2.5136 | Val Loss: 8.6019
Training model with 10 epochs, batch size 64, hidden size 512
Model has 17,277,537 trainable parameters
Training model with 10 epochs, batch size 64, hidden size 512
Sample 5:
Temperature: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5]
Humidity: [-0.3, -0.4, -0.5, -0.6, -0.6, -0.6, -0.7, -0.8, -0.8, -1.0, -1.2, -1.3, -1.3, -1.2, -1.0, -0.9, -0.7, 0.0, 0.3, 0.4, 0.3, -0.0, -0.4, -0.2]
Cloudiness: [-1.13, -1.13, -1.13, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, 0.25, -0.09, -0.44, -0.44, -0.09]
Original: [CLS] In <city> brennt die Sonne am Morgen und Nachmittag. Die Temperatur steigt auf schreckliche 26 <temp>. Die Hitze zieht durch die Straßen. Am Abend zieht eine dunkle Wolkendecke auf. Der Himmel weint trübe Gedanken. Die Temperatur bleibt bei 22 <temp>. Die Nacht bringt Schatten. Wolken verschlingen die Sterne. Ihre Lichtpunkte sind kaum noch zu sehen. Es ist, als ob die Dunkelheit alles verschluckt. Die Temperaturen sinken auf unbehagliche 23 <temp>. Ein unheilvolles Gefühl schleicht durch die Luft. Der Wind trägt leise Schreie mit sich. Der Tag endet im Zwielicht, während [SEP]
Generated: [PAD]lose Sonne am Tag. Die Temperaturen steigen auf 23 bis 36 <temp>. Die Luft flirrt vor Hitze, während die Hitze des Tages schwindet. In der Nacht wird der Himmel klar, doch die Dunkelheit bringt keine Erleichterung. Die Wolken bleiben wie ein schwerer Vorhang aus. Die Temperaturen sinken auf 23 <temp>. Die Nacht bringt keine Erleichterung. Die Stille der Nacht ist erdrückend, als ob die Welt in einen tiefen Schlaf gefallen ist. Kein Wind weht,
--------------------------------------------------------------------------------

Train Loss: 3.3150 | Val Loss: 8.1026
Training model with 10 epochs, batch size 32, hidden size 512
Model has 17,277,537 trainable parameters
Original: [CLS] In <city> erhebt sich der Morgen wie ein schwerer Schleier aus Wolken. Die Temperatur bleibt gefangen bei 5 <temp>. Mit dem Mittag zieht der Regen auf, als wären dunkle Mächte am Werk. Den ganzen Abend über wird das Land in Wasser getränkt, während die Werte zwischen 5 und 7 <temp> schwanken. Der Himmel weint und die Welt bleibt im Schatten. In der Nacht, wenn die Dunkelheit sich ausbreitet, wird der Regen nicht aufhören. Die Temperaturen sinken zurück auf 5 <temp>. Die Luft fühlt sich kalt und feucht an, als ob die Welt in einem ewigen Sturm gefangen ist. Ein unheilvolles [SEP]
Generated: [PAD] am Nachmittag übers. Die Temperaturen steigen von 13 <temp>. Die Luft ist schwer und bedrückt. Die Wolken hängen tief, als ob die Welt. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Kälte kriecht. Die Luft ist schwer. Die Luft ist schwer und bedrückend. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Temperaturen steigen
--------------------------------------------------------------------------------
Done!

Train Loss: 2.1875 | Val Loss: 9.0432
# BASIC Implementation
Params: 
batch:  128
hidden: 256
droput: 0.2
Best loss: 2.2737
8,161 tokens

CAP at 6 M Params
# Loss Function Modification with old params
- Using CrossEntropyLoss with label smoothing 0.1
- Using Focal loss with gama=2 and alpha=0.25
- Achieved Best Loss: 0.5061 (Focal Loss) with Val loss: 2.0764


# Loss Function Modification with new params
- Using CrossEntropyLoss with label smoothing 0.1
- Using Focal loss with gama=2 and alpha=0.25
- Droput: 0.3
- Batch: 32
- Hidden: 256
1.st run CAP at 6 M Params
- Best Loss: 0.5407 | Val Loss: 2.1299
2.nd run CAP at 10 M Params
- Not necessary to run again, the model is not doing better

# crossentropy with a lot of params
Train Loss: 2.5136 | Val Loss: 8.6019
Training model with 10 epochs, batch size 64, hidden size 512
Model has 17,277,537 trainable parameters
Training model with 10 epochs, batch size 64, hidden size 512
Sample 5:
Temperature: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.6, 0.6, 0.6, 0.6, 0.6, 0.5]
Humidity: [-0.3, -0.4, -0.5, -0.6, -0.6, -0.6, -0.7, -0.8, -0.8, -1.0, -1.2, -1.3, -1.3, -1.2, -1.0, -0.9, -0.7, 0.0, 0.3, 0.4, 0.3, -0.0, -0.4, -0.2]
Cloudiness: [-1.13, -1.13, -1.13, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, -0.79, 0.25, -0.09, -0.44, -0.44, -0.09]
Original: [CLS] In <city> brennt die Sonne am Morgen und Nachmittag. Die Temperatur steigt auf schreckliche 26 <temp>. Die Hitze zieht durch die Straßen. Am Abend zieht eine dunkle Wolkendecke auf. Der Himmel weint trübe Gedanken. Die Temperatur bleibt bei 22 <temp>. Die Nacht bringt Schatten. Wolken verschlingen die Sterne. Ihre Lichtpunkte sind kaum noch zu sehen. Es ist, als ob die Dunkelheit alles verschluckt. Die Temperaturen sinken auf unbehagliche 23 <temp>. Ein unheilvolles Gefühl schleicht durch die Luft. Der Wind trägt leise Schreie mit sich. Der Tag endet im Zwielicht, während [SEP]
Generated: [PAD]lose Sonne am Tag. Die Temperaturen steigen auf 23 bis 36 <temp>. Die Luft flirrt vor Hitze, während die Hitze des Tages schwindet. In der Nacht wird der Himmel klar, doch die Dunkelheit bringt keine Erleichterung. Die Wolken bleiben wie ein schwerer Vorhang aus. Die Temperaturen sinken auf 23 <temp>. Die Nacht bringt keine Erleichterung. Die Stille der Nacht ist erdrückend, als ob die Welt in einen tiefen Schlaf gefallen ist. Kein Wind weht,
--------------------------------------------------------------------------------

Train Loss: 3.3150 | Val Loss: 8.1026
Training model with 10 epochs, batch size 32, hidden size 512
Model has 17,277,537 trainable parameters
Original: [CLS] In <city> erhebt sich der Morgen wie ein schwerer Schleier aus Wolken. Die Temperatur bleibt gefangen bei 5 <temp>. Mit dem Mittag zieht der Regen auf, als wären dunkle Mächte am Werk. Den ganzen Abend über wird das Land in Wasser getränkt, während die Werte zwischen 5 und 7 <temp> schwanken. Der Himmel weint und die Welt bleibt im Schatten. In der Nacht, wenn die Dunkelheit sich ausbreitet, wird der Regen nicht aufhören. Die Temperaturen sinken zurück auf 5 <temp>. Die Luft fühlt sich kalt und feucht an, als ob die Welt in einem ewigen Sturm gefangen ist. Ein unheilvolles [SEP]
Generated: [PAD] am Nachmittag übers. Die Temperaturen steigen von 13 <temp>. Die Luft ist schwer und bedrückt. Die Wolken hängen tief, als ob die Welt. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Kälte kriecht. Die Luft ist schwer. Die Luft ist schwer und bedrückend. Die Temperaturen steigen auf 13 <temp>. Die Luft ist schwer und bedrückend. Die Temperaturen steigen
--------------------------------------------------------------------------------
Done!

Train Loss: 2.1875 | Val Loss: 9.0432
Training model with 10 epochs, batch size 128, hidden size 512
Model has 17,277,537 trainable parameters
Original: [CLS] In <city> hält der Himmel seinen Atem an. Tagsüber kämpfen Sonne und Wolken um die Vorherrschaft. Die Temperatur strebt nach 26 <temp> und erinnert an alte Zeiten. Am Himmel braut sich Unheil zusammen, während der Wind flüstert. Nachts, wenn die Dunkelheit über die Stadt fällt, bildet sich ein leichter Schleier aus Wolken. Die Luft wird dünn und schwer, während die Temperaturen auf die tiefsten Tiefen von 26 <temp> sinken. Ein Gefühl von Chaos schwebt in der Stille. Unter dem grauen Zelt der Nacht ruhen die Geheimnisse der Welt. In dieser Nacht wird der Himmel zur Leinwand für das [SEP]
Generated: [PAD] Sonnenschein. Die Luft ist warm mit Temperaturen zwischen 21 und 29 <temp>. Die Luft flirrt und die Luft wird schwer. In der Nacht zieht der Himmel auf. Die Sterne blitzen wie ferne Augen. Die Temperatur sinkt auf 23 <temp>. Die Dunkelheit bringt keine Ruhe. Der Himmel bleibt klar und die Sterne blitzen wie ferne Augen. Die Sterne blitzen wie ferne Augen, die die Sterne blitzen wie ferne Augen. Die Temperatur sinkt auf 22 <temp>. Die Nacht
--------------------------------------------------------------------------------


Model has 20,693,598 trainable parameters
Training model with 20 epochs, batch size 128, hidden size 512
Loss 2.6473
Final generated samples:

Sample 1:
Temperature: [0.9, 0.9, 0.8, 0.8, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]
Humidity: [0.5, 0.5, 0.7, 0.7, 0.4, 0.4, 0.4, 0.4, 0.4, 0.3, 0.3, 0.4, 0.4, 0.3, 0.3, 0.3, 0.4, 0.4, 0.5, 0.5, 0.5, 0.4, 0.4, 0.4]
Cloudiness: [0.16, 0.5, 0.5, 0.5, 0.5, 0.84, 0.84, 1.51, 0.84, 0.5, 0.16, 0.16, 0.5, 0.84, 0.84, 0.5, 0.5, 0.5, 0.5, 0.16, -0.17, 0.16, 0.5, 0.5]
Original: [CLS] In <city> zieht die Dunkelheit auf. Regen fällt unaufhörlich vom düsteren Himmel. Die Luft ist feucht und die Temperaturen steigen nur leicht. Sie bewegen sich zwischen 27 und 28 <temp>, während die Wolken die Sonne verschlingen. Die Nacht bricht herein und bringt mehr Regen. Die Kälte kommt mit der Dämmerung. Die Temperaturen sinken nicht, sie bleiben starr bei 27 <temp>. Die Tropfen fallen schwer auf den Boden und die Geräusche der Natur verschwinden. Es ist, als ob der Tag und die Nacht sich in einem ständigen Kampf befinden. Die Welt fühlt sich anders an, als ob die Elemente die Kontrolle übernommen [SEP]
Generated: In <city> zieht der Zorn des Himmels auf. Tagsüber fallen schwere Tropfen vom Himmel. Die Temperaturen schwanken zwischen 26 und 27 <temp>. Die Luft fühlt sich schwer an, als ob sie die Welt erdrücken will. In der Nacht wird der Regen nicht endenhören. Die Dunkelheit bringt keine Ruhe. Die Tiefstwerte erreichen 26 <temp>. Die Luft ist schwer und drückend. Die Tropfen fallen wie Tränen vom Himmel. Die Luft ist schwer und feucht. Die Wolken scheinen zu
--------------------------------------------------------------------------------

Sample 2:
Temperature: [0.5, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4, 0.6, 0.8, 1.1, 1.3, 1.5, 1.6, 1.6, 1.6, 1.5, 1.2, 1.0, 0.8, 0.8, 0.7, 0.7, 0.6, 0.5]
Humidity: [0.6, 0.6, 0.6, 0.9, 1.0, 1.0, 1.0, 0.5, -0.1, -0.7, -1.1, -1.4, -1.6, -1.6, -1.7, -1.5, -0.9, -0.6, -0.3, 0.1, 0.3, 0.3, 0.5, 0.7]
Cloudiness: [-1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -1.02, -0.68, -0.34, 0.0, 1.7, 0.34, 0.0, 0.0, 0.0, -0.34]
Original: [CLS] In <city> beginnt der Tag in feuriger Stille. Die Sonne brennt gnadenlos und die Luft flüstert von 22 bis 35 <temp>. Vom Morgen bis zum Nachmittag ist der Himmel leer, wie ein endloses Meer aus Licht. Am Abend kommt ein Hauch von Wolken, doch die Hitze bleibt. Die Temperaturen schweben zwischen 25 und 27 <temp>, während die Dunkelheit naht. In der Nacht entfaltet sich der schlafende Himmel. Keine Wolken trüben den Blick auf die funkelnden Sterne. Die Temperaturen sinken auf 22 <temp>, während das Licht der fernen Welten die ruhige Nacht küss [SEP]
Generated: In <city> bricht der Tag an mit strahlendem Licht. Die Sonne brennt gnadenlos und die Temperaturen steigen auf gefährliche Werte zwischen 23 und 34 <temp>. Am Abend zieht ein Schatten über <city>. Der Himmel ist klar und die Temperaturen sinken auf 24 bis 27 <temp>. Die Dunkelheit bringt eine unheimliche Stille. In der Nacht bleibt der Himmel klar und unberührt. Die Sterne blitzen wie ferne Augen im Dunkelheit. Die Temperatur sinkt auf 23 <temp>, doch die Stille der Nacht
--------------------------------------------------------------------------------

Sample 3:
Temperature: [-0.7, -0.7, -0.7, -0.7, -0.7, -0.8, -0.5, -0.4, -0.2, -0.0, 0.1, 0.1, 0.1, 0.1, 0.0, -0.1, -0.2, -0.3, -0.4, -0.4, -0.4, -0.5, -0.5, -0.5]
Humidity: [0.5, 0.5, 0.4, 0.3, 0.2, 0.4, -0.1, -0.3, -0.6, -0.9, -1.1, -1.0, -1.1, -1.0, -0.8, -0.3, 0.0, 0.2, 0.4, 0.4, 0.3, 0.5, 0.5, 0.5]
Cloudiness: [-0.66, -0.66, -0.66, -0.66, -0.66, -0.66, -0.66, -0.66, -0.66, -0.66, -0.66, -0.32, 0.02, 1.72, 1.72, 1.72, 1.72, 1.72, 1.72, 1.72, 1.72, 1.72, 1.72, 1.72]
Original: [CLS] In <city> herrscht am Morgen eine unberührte Leere. Der Himmel ist frei von Wolken, die Sonne brennt bei frostigen 8 <temp>. Doch der Mittag bringt das Unheil ; Regenwolken sammeln sich und der Himmel weint, während die Temperatur auf 19 <temp> ansteigt. Am Abend hüllt sich <city> in eine dunkle, dichte Decke aus Wolken. Das Nassen bleibt jedoch fern, die Luft fühlt sich kühl an bei 12 bis 13 <temp>. Doch die Nacht bringt keine Erlösung. Die Wolken bleiben, als ob sie uns gefangen halten. Die Werte stagnieren bei 12 <temp>. Eine stille Dunkel [SEP]
Generated: In <city> bricht der Tag an mit strahlendem Licht und Schatten. Die Sonne strahlt hell und die Temperatur liegt bei 13 <temp>. Doch der Frieden währt nicht lange. Ab dem Mittag zieht ein Sturm auf. Regen fällt unaufhörlich, während die Temperaturen auf 16 <temp> steigen. Der Himmel wird grau und trüb. Am Abend hüllt sich <city> in einen grauen Schleier aus Wolken, die die Temperaturen auf 13 bis 15 <temp> sinken. Die Dunkelheit bringt eine unheimliche Still
--------------------------------------------------------------------------------

Sample 4:
Temperature: [-1.6, -1.6, -1.6, -1.6, -1.6, -1.6, -1.6, -1.6, -1.6, -1.6, -1.5, -1.4, -1.3, -1.3, -1.3, -1.3, -1.3, -1.3, -1.3, -1.3, -1.3, -1.3, -1.3, -1.3]
Humidity: [0.8, 0.9, 0.9, 0.8, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.8, 0.6, 0.4, 0.4, 0.5, 0.6, 0.7, 0.8, 0.8, 0.9, 0.9, 1.0, 1.1, 1.1]
Cloudiness: [0.21, -0.13, -0.13, -0.46, 0.21, -0.13, -0.13, -1.13, -0.46, -0.13, -0.46, -0.46, -0.13, 0.21, 0.87, 1.21, 1.54, 1.21, 1.54, 1.54, 1.54, 1.54, 1.54, 1.54]
Original: [CLS] In <city> zeigt sich am Morgen der Himmel in dunklem Blau, nur wenige Wolken ziehen vorbei. Die Temperatur beginnt bei frischen 3 <temp>. Im Laufe des Tages klettert das Thermometer auf 6 <temp>. Doch wenn die Dunkelheit einbricht, ergreift das Grau die Stadt. Nebel rollt herein und hüllt alles ein. Die Nacht wird kalt und unheimlich, mit Tiefstwerten von 6 <temp>. Der Nebel frisst sich durch die Strassen und macht die Sicht fast unmöglich. Ein Gefühl von Gefahr und Unheimlichkeit liegt in der Luft. Nur die Schatten der Bäume ragen aus dem dicht [SEP]
Generated: In <city> bricht der Tag an mit strahlendem Licht. Die Sonne strahlt hell und die Kälte bei 1 <temp> lässt die Luft gefriert. Doch der Frieden währt nicht lange. Am Nachmittag und Abend ziehen dunkle Wolken auf, und die Temperaturen steigen nur leicht auf 4 bis 5 <temp>. Die Dunkelheit bringt keine Erleichterung. In der Nacht wird der Himmel noch dunkler. Die Wolken bleiben dicht und schwer, und die Kälte kriecht in jede Ecke. Die Tiefsttemperaturen erreichen 4
--------------------------------------------------------------------------------

Sample 5:
Temperature: [-0.2, -0.2, -0.2, -0.3, -0.3, -0.3, -0.3, -0.1, 0.1, 0.3, 0.5, 0.5, 0.5, 0.5, 0.4, 0.3, 0.1, -0.2, -0.2, -0.3, -0.3, -0.3, -0.3, -0.3]
Humidity: [0.5, 0.5, 0.4, 0.6, 0.6, 0.9, 0.7, 0.2, -0.1, -0.5, -0.9, -0.9, -0.9, -0.8, -0.6, -0.4, 0.1, 0.6, 0.7, 0.8, 0.7, 0.7, 0.6, 0.8]
Cloudiness: [-0.74, 0.33, 1.04, 1.04, 1.04, 1.04, -0.38, 0.33, -0.03, -0.38, -0.38, -0.38, -1.1, -1.1, -1.1, -1.1, -1.1, -1.1, -1.1, -1.1, -0.74, -0.74, -0.74, -0.74]
Original: [CLS] In <city> bricht der Tag an mit einer Mischung aus Sonne und Wolken. Die Temperaturen erreichen 16 <temp>. Zur Mittagszeit wird die Sonne stark und zeigt ihre Macht bei 25 <temp>. Am Abend ziehen düstere Wolken durch den Himmel, doch der blaue Grund bleibt sichtbar. Die Temperatur sinkt auf 17 bis 18 <temp>. In der Nacht hüllen Wolken den Himmel in ihre dunklen Schatten und das Thermometer zeigt 16 <temp>. Ein Gefühl der Beklemmung liegt in der Luft. Stille breitet sich aus, während die Wolken sich versammeln und das Licht der Sterne verdecken. Flüchtige Schatten huschen vorbei und künden von [SEP]
Generated: In <city> zieht der Morgen düstere Wolken auf. Die Sonne kämpft gegen die Wolken und die Temperatur liegt bei 16 <temp>. Im Laufe des Tages bricht die Sonne durch die Wolken und die Hitze steigt auf 28 <temp>. Am Abend zeigt sich der Himmel klar und unberührt. Die Temperaturen sinken auf 17 bis 19 <temp>. In der Nacht wird der Himmel klar und unberührt. Die Sterne blitzen wie ferne Augen im Dunkelheit. Die Kälte schleicht sich heran, die Tiefsttemperaturen erreichen 17 <temp>.
--------------------------------------------------------------------------------
Done!


Training model with 20 epochs, batch size 128, hidden size 256
Using device: cuda
Loading tokenizer...
Preparing datasets...
Dataset preparation completed in 0.00s
Analyzing vocabulary usage to reduce model size...
Analyzing vocabulary usage to reduce model size...
Scanning token usage: 100%|██████████| 19317/19317 [00:59<00:00, 322.44it/s]
Reduced vocabulary from 30,000 to 8,158 tokens (27.2%)
Feature dimension: 18
Reduced vocabulary size: 8158
Training samples: 17385
Validation samples: 1932
Creating model...
Model has 8,319,262 trainable parameters

Loss: 2.9484
Sample 3:
Temperature: [-0.8, -0.8, -0.8, -0.8, -0.8, -0.9, -0.7, -0.5, -0.3, -0.1, 0.1, 0.1, 0.1, 0.1, 0.0, -0.2, -0.3, -0.4, -0.5, -0.5, -0.5, -0.6, -0.6, -0.6]
Humidity: [0.4, 0.4, 0.3, 0.2, 0.2, 0.3, -0.1, -0.4, -0.7, -1.0, -1.1, -1.1, -1.1, -1.1, -0.9, -0.4, -0.1, 0.1, 0.3, 0.3, 0.2, 0.4, 0.4, 0.4]
Cloudiness: [-0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.8, -0.46, -0.12, 1.57, 1.57, 1.57, 1.57, 1.57, 1.57, 1.57, 1.57, 1.57, 1.57, 1.57]
Original: [CLS] In <city> herrscht am Morgen eine unberührte Leere. Der Himmel ist frei von Wolken, die Sonne brennt bei frostigen 8 <temp>. Doch der Mittag bringt das Unheil ; Regenwolken sammeln sich und der Himmel weint, während die Temperatur auf 19 <temp> ansteigt. Am Abend hüllt sich <city> in eine dunkle, dichte Decke aus Wolken. Das Nassen bleibt jedoch fern, die Luft fühlt sich kühl an bei 12 bis 13 <temp>. Doch die Nacht bringt keine Erlösung. Die Wolken bleiben, als ob sie uns gefangen halten. Die Werte stagnieren bei 12 <temp>. Eine stille Dunkel [SEP]
...
Original: [CLS] In <city> drückt ein schwerer Himmel die Stimmung. Morgens ist die Sonne verloren, die Kälte greift mit 13 <temp> nach den Menschen. Ein kurzer Hoffnungsschimmer, später lockert die Dunkelheit die Wolken etwas. Es erreicht die 22 <temp>, doch die Freude bleibt flüchtig. Am Abend sinkt die Nacht über <city> wie ein schwerer Vorhang. Die Temperaturen fallen auf 14 bis 15 <temp>. Der Himmel wird erneut zum düsteren Gefangenen, kaum Licht durchbricht die grübelnden Wolken. Die Nacht umhüllt die Stadt in einem undurchdringlichen Grau. Die Luft wird kühl und drückend, besiegt bei tief [SEP]
Generated: In <city> zieht der Morgen düster und trüb. Die Sonne kämpft vergeblich gegen die Wolken, während die Temperatur bei 12 <temp> verweilt. Im Laufe des Tages zieht ein dunkler Schleier über den Himmel. Die Sonne kämpft sich durch die Wolken, während die Temperaturen auf 16 <temp> steigen. Am Abend zieht ein dunkler Schleier über <city>, die die Temperaturen sinken auf 13 bis 15 <temp>. Die Nacht bringt eine unheimliche Stille. Der Himmel bleibt bedeckt und die Kälte kriecht in die Herzen
--------------------------------------------------------------------------------
